# IOT-IPS

The script for iot-ips project

Current use a simple fully-connected model with rssi information only.

If you want to add more information as input, you can modify the parser function.

In [1]:
import os
import sys
import csv
import math
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

/home/rtu/anaconda3/envs/habitat/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

## Define the input format

In [2]:
def parser(rssi, csi, position):
    '''
    description:
        this is a function to rearrange the raw data into the form of your model input
        so that you can easily design the input format and arrange the input data.
        
    input:
        rssi: the averaged rssi in one collect file, stored in numpy array
        csi: an array consists of all csi data in one collect file, stored in a 2D numpy array
        position: nparray(x,y)
        Note: both the rssi and csi belong to the same location
        
    output:
        x: the list containing inputs of the model
        y: the correspond ground truth
    '''
    
    x = [rssi,]
    y = [position,]
    
    return x, y # use rssi data only as example

## Dataset

In [3]:
class Dataset(Dataset):
    
    def __init__(self, path, parser):
        filenames = os.listdir(path)
        self.parser = parser
        
        self.x = []
        self.y = []
        
        for name in filenames:
            if 'csi' in name:
                file = os.path.join(path, name)
                x,y = self.readOne(file)
                self.x.extend(x)
                self.y.extend(y)
                      
    def readOne(self, fname):
        
        if 'csi' in fname:
            csi = fname
            rssi = fname.replace('csi','rssi')
        elif 'rssi' in fname:
            rssi = fname
            csi = fname.replace('rssi','csi')
        
        # parse the position from the file name
        position = csi[csi.find('csi')+3: csi.rfind('.')].split('_')
        position = np.asfarray([float(i) for i in position])
        
        rssi_f = open(rssi)
        csi_f = open(csi)
        rssilines = rssi_f.readlines()
        csilines = csi_f.readlines()
        
        # Read the rssi and average
        row = 0
        rssi_data = np.zeros(8, dtype=float)
        for line in rssilines:
            rssi_raw = line.split(',')
            try:
                rssi_data += np.asfarray([int(i) for i in rssi_raw])
                row += 1
            except:
                continue
        rssi_data /= row
            
        # Read the csi
        csi_data = []
        for line in csilines:
            csi_raw = line.split(',')[-2][1:-2]  # skip the brackets and space
            csi_str = csi_raw.split(' ')
            csi_float = np.asfarray([int(i) for i in csi_str])
            csi_data.append(csi_float)
        csi_data = np.asfarray(csi_data)
        
        rssi_f.close()
        csi_f.close()
        
        x,y = self.parser(rssi_data, csi_data, position)
        x = [torch.Tensor(i) for i in x]
        y = [torch.Tensor(i) for i in y]
        
        return x,y
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Model

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        layers = [
            nn.Linear(8,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,2),
        ]
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

# Hyper Parameters

In [5]:
lr = 0.0001
momentum = 0.01
n_epochs = 1500
batch_size = 32
valid_epoch = 100
save_path = './test.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
train_dataset = Dataset('data/train', parser)
test_dataset = Dataset('data/test', parser)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

model = Model().to(device)

In [7]:
criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum) 


best_loss, step, early_stop_count = math.inf, 0, 0

train_pbar = tqdm(range(n_epochs), position=0, leave=True)
for epoch in train_pbar:
    model.train()
    loss_record = []

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)             
        loss = criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

    if epoch % valid_epoch==0:
        mean_train_loss = sum(loss_record)/len(loss_record)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

    if mean_valid_loss < best_loss:
        best_loss = mean_valid_loss
        torch.save(model.state_dict(), save_path)
        print('Saving model with loss {:.3f}...'.format(best_loss))


Epoch [55/1500]:   3%|▍           | 52/1500 [00:00<00:05, 265.96it/s, loss=2.82]

Epoch [1/1500]: Train loss: 13.2379, Valid loss: 40.6984
Saving model with loss 40.698...


Epoch [153/1500]:   9%|▉         | 134/1500 [00:00<00:05, 266.10it/s, loss=2.52]

Epoch [101/1500]: Train loss: 2.6522, Valid loss: 6.2970
Saving model with loss 6.297...


Epoch [259/1500]:  16%|█▋        | 245/1500 [00:00<00:04, 273.52it/s, loss=2.31]

...

Epoch [1361/1500]:  89%|███████▏| 1341/1500 [00:05<00:00, 286.31it/s, loss=1.37]

Epoch [1301/1500]: Train loss: 1.3526, Valid loss: 4.4453


Epoch [1449/1500]:  95%|████████▌| 1427/1500 [00:05<00:00, 259.75it/s, loss=1.3]

Epoch [1401/1500]: Train loss: 1.3368, Valid loss: 4.4246


Epoch [1500/1500]: 100%|████████| 1500/1500 [00:05<00:00, 266.77it/s, loss=1.29]


In [8]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

model.eval() # Set your model to evaluation mode.
loss_record = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        print(f'Input: {x.cpu().detach().numpy()[0]}')
        print(f'Prediction: {pred.cpu().detach().numpy()[0]}')
        print(f'GT        : {y.cpu().detach().numpy()[0]}\n\n')

Input: [-27. -33. -45. -36. -41. -45. -55. -45.]
Prediction: [0.7224847 3.9309115]
GT        : [1. 7.]


Input: [-37. -35. -44. -37. -40. -46. -55. -46.]
Prediction: [0.73290074 5.2023344 ]
GT        : [-1.  6.]


Input: [-23. -39. -38. -37. -34. -52. -59. -49.]
Prediction: [1.6801782 2.4403706]
GT        : [4. 3.]


Input: [-32. -37. -50. -35. -40. -44. -60. -51.]
Prediction: [0.84700453 4.900565  ]
GT        : [1. 9.]


Input: [-31. -40. -47. -40. -42. -54. -57. -53.]
Prediction: [0.9110791 5.3505487]
GT        : [1.5 5.5]


Input: [-30. -67. -46. -35. -37. -47. -59. -50.]
Prediction: [1.7826805 2.9247172]
GT        : [-1.  9.]


